In [1]:
from datetime_periods import period

def date_within_period(date, start_date, end_date):
    return (date >= (start_date - timedelta(days=1)) and date <= (end_date + timedelta(days=1)))

def period_within_period(start_date_1, end_date_1, start_date_2, end_date_2):
    return date_within_period(start_date_1, start_date_2, end_date_2) and date_within_period(end_date_1, start_date_2, end_date_2)

def periods_overlap(start_date_1, end_date_1, start_date_2, end_date_2):
    return date_within_period(start_date_1, start_date_2, end_date_2) or date_within_period(end_date_1, start_date_2, end_date_2)

def add_periods(start_date_1, end_date_1, start_date_2, end_date_2):
    if period_within_period(start_date_1, end_date_1, start_date_2, end_date_2):
        return start_date_2, end_date_2
    if period_within_period(start_date_2, end_date_2, start_date_1, end_date_1):
        return start_date_1, end_date_1
    
    dates = [start_date_1, end_date_1, start_date_2, end_date_2]
    min_date = min(dates)
    
    return [min(dates), max(dates)]

def merge_period_into_periods(periods, start_date, end_date):
    if len(periods) == 0:
        periods.append([start_date, end_date])
        return periods
    
    added = False
    
    for period in periods:
        if period_within_period(start_date, end_date, period[0], period[1]):
            added = True
            break
        elif period_within_period(period[0], period[1], start_date, end_date):
            periods.remove(period)
            periods.append([start_date, end_date])
            added = True
            break
        elif periods_overlap(start_date, end_date, period[0], period[1]):
            periods.remove(period)
            periods.append(add_periods(start_date, end_date, period[0], period[1]))
            added = True
            break
    
    if not added:
        periods.append([start_date, end_date])
            
    return periods

In [2]:
import csv
from datetime import date, timedelta, datetime

tsv_file = open("export.tsv")
read_tsv = csv.reader(tsv_file, delimiter="\t")

active_periods_of_person = {}

firstline = True

index = 0
for row in read_tsv:
    if firstline:
        firstline = False
        continue
    
    index += 1
    if index % 1000 == 0:
        print("\r", "Processed", index, "records...", end='')
    
    student_id = row[0]
    course_id = row[1]
    
    start_date = datetime.strptime(row[2], '%y/%m/%d')
    if row[3] == '':
        end_date = datetime.today()
    else:
        end_date = datetime.strptime(row[3], '%y/%m/%d')
    
    if student_id not in active_periods_of_person.keys():
        active_periods_of_person[student_id] = {}
        
    if "courses" not in active_periods_of_person[student_id]:
         active_periods_of_person[student_id]["courses"] = {}
    
    if course_id not in active_periods_of_person[student_id]["courses"].keys():
        active_periods_of_person[student_id]["courses"][course_id] = []
        
    active_periods_of_person[student_id]["courses"][course_id] = merge_period_into_periods(
        active_periods_of_person[student_id]["courses"][course_id], 
        start_date, 
        end_date)

 Processed 500000 records...119000 records...288000 records...Processed 342000 records...367000 records... 409000 records...records...records...records...

In [3]:
active_periods_of_person

{'1': {'courses': {'17484': [[datetime.datetime(2014, 10, 1, 0, 0),
     datetime.datetime(2020, 12, 18, 16, 55, 45, 619372)]],
   '4279': [[datetime.datetime(2014, 10, 1, 0, 0),
     datetime.datetime(2017, 12, 8, 0, 0)]],
   '1': [[datetime.datetime(2011, 9, 26, 0, 0),
     datetime.datetime(2012, 10, 24, 0, 0)]],
   '25514': [[datetime.datetime(2018, 10, 1, 0, 0),
     datetime.datetime(2020, 12, 18, 16, 55, 45, 619372)]],
   '10253': [[datetime.datetime(2010, 10, 1, 0, 0),
     datetime.datetime(2013, 7, 5, 0, 0)]],
   '14101': [[datetime.datetime(2018, 10, 1, 0, 0),
     datetime.datetime(2020, 12, 18, 16, 55, 45, 619372)]]}},
 '2': {'courses': {'465': [[datetime.datetime(2014, 2, 24, 0, 0),
     datetime.datetime(2015, 4, 22, 0, 0)]],
   '16412': [[datetime.datetime(2011, 9, 26, 0, 0),
     datetime.datetime(2014, 2, 23, 0, 0)]],
   '7007': [[datetime.datetime(2018, 12, 10, 0, 0),
     datetime.datetime(2019, 11, 19, 0, 0)],
    [datetime.datetime(2017, 7, 24, 0, 0),
     datetim

In [4]:
def merge_periods(periods):
    if len(periods) == 0:
        return periods
    
    periods = [list(x) for x in {(tuple(e)) for e in periods}]
    
    old_periods = []
    merged_periods = []
    while old_periods != periods:
        old_periods = periods
        for period in periods:
            tmp_periods = periods
            merged_periods = merge_period_into_periods(tmp_periods, period[0], period[1])
        
        periods = merged_periods
        
    return periods

# https://stackoverflow.com/a/48754702
def count_months(start_date,end_date):
    qty_month = ((end_date.year - start_date.year) * 12) + (end_date.month - start_date.month)

    d_days = end_date.day - start_date.day

    if d_days >= 0:
        adjust = 0
    else:
        adjust = -1
    qty_month += adjust

    return qty_month

In [5]:
for student_id, course_periods in active_periods_of_person.items():
    for course_id, periods in active_periods_of_person[student_id]["courses"].items():
        periods = merge_periods(periods)
    
for student_id, data in active_periods_of_person.items():
    active_periods_of_person[student_id]["All courses periods"] = []    
    for course_id, periods in active_periods_of_person[student_id]["courses"].items():
        active_periods_of_person[student_id]["All courses periods"] += periods

    active_periods_of_person[student_id]["All courses periods"] = merge_periods(active_periods_of_person[student_id]["All courses periods"])
    
    student_months_total = 0
    for period in active_periods_of_person[student_id]["All courses periods"]:
        student_months_total += count_months(period[0], period[1])
    
    active_periods_of_person[student_id]["Total months"] = student_months_total

active_periods_of_person

{'1': {'courses': {'17484': [[datetime.datetime(2014, 10, 1, 0, 0),
     datetime.datetime(2020, 12, 18, 16, 55, 45, 619372)]],
   '4279': [[datetime.datetime(2014, 10, 1, 0, 0),
     datetime.datetime(2017, 12, 8, 0, 0)]],
   '1': [[datetime.datetime(2011, 9, 26, 0, 0),
     datetime.datetime(2012, 10, 24, 0, 0)]],
   '25514': [[datetime.datetime(2018, 10, 1, 0, 0),
     datetime.datetime(2020, 12, 18, 16, 55, 45, 619372)]],
   '10253': [[datetime.datetime(2010, 10, 1, 0, 0),
     datetime.datetime(2013, 7, 5, 0, 0)]],
   '14101': [[datetime.datetime(2018, 10, 1, 0, 0),
     datetime.datetime(2020, 12, 18, 16, 55, 45, 619372)]]},
  'All courses periods': [[datetime.datetime(2010, 10, 1, 0, 0),
    datetime.datetime(2013, 7, 5, 0, 0)],
   [datetime.datetime(2011, 9, 26, 0, 0),
    datetime.datetime(2012, 10, 24, 0, 0)],
   [datetime.datetime(2014, 10, 1, 0, 0),
    datetime.datetime(2020, 12, 18, 16, 55, 45, 619372)],
   [datetime.datetime(2018, 10, 1, 0, 0),
    datetime.datetime(2020

In [6]:
import json

def date_to_timestamp(d):
    return int(d.timestamp())

for student_id, data in active_periods_of_person.items():
    for course_id, periods in active_periods_of_person[student_id]["courses"].items():
        active_periods_of_person[student_id]["courses"][course_id] = [[date_to_timestamp(date[0]), date_to_timestamp(date[1])] for date in periods]  
        
    active_periods_of_person[student_id]["All courses periods"] = [
        [date_to_timestamp(date[0]), date_to_timestamp(date[1])] for date in active_periods_of_person[student_id]["All courses periods"]]  

In [7]:
active_periods_of_person

{'1': {'courses': {'17484': [[1412114400, 1608306945]],
   '4279': [[1412114400, 1512687600]],
   '1': [[1316988000, 1351029600]],
   '25514': [[1538344800, 1608306945]],
   '10253': [[1285884000, 1372975200]],
   '14101': [[1538344800, 1608306945]]},
  'All courses periods': [[1285884000, 1372975200],
   [1316988000, 1351029600],
   [1412114400, 1608306945],
   [1538344800, 1608306945],
   [1412114400, 1512687600]],
  'Total months': 183},
 '2': {'courses': {'465': [[1393196400, 1429653600]],
   '16412': [[1316988000, 1393110000]],
   '7007': [[1544396400, 1574118000], [1500847200, 1542927600]]},
  'All courses periods': [[1500847200, 1542927600],
   [1544396400, 1574118000],
   [1316988000, 1429653600],
   [1316988000, 1429653600]],
  'Total months': 110},
 '3': {'courses': {'20707': [[1538344800, 1608306945]],
   '13118': [[1316988000, 1424386800]],
   '10846': [[1427666400, 1476309600]],
   '20161': [[1349042400, 1540940400]],
   '19626': [[1538344800, 1608306945]]},
  'All courses

In [11]:
with open('Studenci-kursy-daty.json', 'w', encoding='utf-8') as f:
    json.dump(active_periods_of_person, f, ensure_ascii=False, indent=2)